In [1]:
import pandas as pd
import os
import datetime
import pickle
import re
import time

In [2]:
# All unique articles
df=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Jan1985-Dec2021/RegSentsExpand_NounChunks.pkl')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990262 entries, 0 to 990261
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           990262 non-null  object        
 1   Title                        990262 non-null  object        
 2   Type                         990262 non-null  object        
 3   StartDate                    990262 non-null  datetime64[ns]
 4   EndDate                      990262 non-null  object        
 5   TextWordCount                990262 non-null  object        
 6   PubTitle                     990262 non-null  object        
 7   SourceType                   990262 non-null  object        
 8   Year                         990262 non-null  float64       
 9   Month                        990262 non-null  float64       
 10  Newspaper                    990262 non-null  object        
 11  RegSentsExpand            

In [3]:
# Use filtered noun chunk matches to define reg relevance
df.loc[df['NounChunkMatchFiltered']!=0,'RegRelevance']=1
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990262 entries, 0 to 990261
Data columns (total 16 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           990262 non-null  object        
 1   Title                        990262 non-null  object        
 2   Type                         990262 non-null  object        
 3   StartDate                    990262 non-null  datetime64[ns]
 4   EndDate                      990262 non-null  object        
 5   TextWordCount                990262 non-null  object        
 6   PubTitle                     990262 non-null  object        
 7   SourceType                   990262 non-null  object        
 8   Year                         990262 non-null  float64       
 9   Month                        990262 non-null  float64       
 10  Newspaper                    990262 non-null  object        
 11  RegSentsExpand            

In [5]:
# LM uncertainty scores
LMuncertainty=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Jan1985-Dec2021/LMuncertainty.csv')
print(LMuncertainty.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990262 entries, 0 to 990261
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   ID                990262 non-null  int64 
 1   UncertaintyCount  990262 non-null  int64 
 2   UncertaintyWords  990262 non-null  object
dtypes: int64(2), object(1)
memory usage: 22.7+ MB
None


In [6]:
LMuncertainty['ID']=LMuncertainty['ID'].astype('str')
print(LMuncertainty.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990262 entries, 0 to 990261
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   ID                990262 non-null  object
 1   UncertaintyCount  990262 non-null  int64 
 2   UncertaintyWords  990262 non-null  object
dtypes: int64(1), object(2)
memory usage: 22.7+ MB
None


In [7]:
# Merge
df2=df.merge(LMuncertainty,on='ID',how='right')
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990262 entries, 0 to 990261
Data columns (total 18 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           990262 non-null  object        
 1   Title                        990262 non-null  object        
 2   Type                         990262 non-null  object        
 3   StartDate                    990262 non-null  datetime64[ns]
 4   EndDate                      990262 non-null  object        
 5   TextWordCount                990262 non-null  object        
 6   PubTitle                     990262 non-null  object        
 7   SourceType                   990262 non-null  object        
 8   Year                         990262 non-null  float64       
 9   Month                        990262 non-null  float64       
 10  Newspaper                    990262 non-null  object        
 11  RegSentsExpand            

In [8]:
# GI sentiments
GIsentiments=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Jan1985-Dec2021/GIsentiments.csv')
GIsentiments['ID']=GIsentiments['ID'].astype('str')
print(GIsentiments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990262 entries, 0 to 990261
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ID              990262 non-null  object
 1   TotalWordCount  990262 non-null  int64 
 2   GIposCount      990262 non-null  int64 
 3   GInegCount      990262 non-null  int64 
 4   GIposWords      990262 non-null  object
 5   GInegWords      990262 non-null  object
dtypes: int64(3), object(3)
memory usage: 45.3+ MB
None


In [9]:
# LSD sentiments
LSDsentiments=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Jan1985-Dec2021/LSDsentiments.csv')
LSDsentiments['ID']=LSDsentiments['ID'].astype('str')
print(LSDsentiments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990262 entries, 0 to 990261
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ID           990262 non-null  object
 1   LSDposCount  990262 non-null  int64 
 2   LSDnegCount  990262 non-null  int64 
 3   LSDposWords  990262 non-null  object
 4   LSDnegWords  990262 non-null  object
dtypes: int64(2), object(3)
memory usage: 37.8+ MB
None


In [10]:
# LM sentiments
LMsentiments=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Jan1985-Dec2021/LMsentiments.csv')
LMsentiments['ID']=LMsentiments['ID'].astype('str')
print(LMsentiments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990262 entries, 0 to 990261
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ID          990262 non-null  object
 1   LMposCount  990262 non-null  int64 
 2   LMnegCount  990262 non-null  int64 
 3   LMposWords  990262 non-null  object
 4   LMnegWords  990262 non-null  object
dtypes: int64(2), object(3)
memory usage: 37.8+ MB
None


In [11]:
# Merge
df3=df2.merge(GIsentiments,on='ID',how='left').\
    merge(LMsentiments,on='ID',how='left').\
    merge(LSDsentiments,on='ID',how='left')
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990262 entries, 0 to 990261
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           990262 non-null  object        
 1   Title                        990262 non-null  object        
 2   Type                         990262 non-null  object        
 3   StartDate                    990262 non-null  datetime64[ns]
 4   EndDate                      990262 non-null  object        
 5   TextWordCount                990262 non-null  object        
 6   PubTitle                     990262 non-null  object        
 7   SourceType                   990262 non-null  object        
 8   Year                         990262 non-null  float64       
 9   Month                        990262 non-null  float64       
 10  Newspaper                    990262 non-null  object        
 11  RegSentsExpand            

In [13]:
# Calculate scores
df3['UncertaintyScore']=df3['UncertaintyCount']/df3['TotalWordCount']*100
for dic in ['GI','LSD','LM']:
    df3[dic+'score']=(df3[dic+'posCount']-df3[dic+'negCount'])/df3['TotalWordCount']*100
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990262 entries, 0 to 990261
Data columns (total 35 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           990262 non-null  object        
 1   Title                        990262 non-null  object        
 2   Type                         990262 non-null  object        
 3   StartDate                    990262 non-null  datetime64[ns]
 4   EndDate                      990262 non-null  object        
 5   TextWordCount                990262 non-null  object        
 6   PubTitle                     990262 non-null  object        
 7   SourceType                   990262 non-null  object        
 8   Year                         990262 non-null  float64       
 9   Month                        990262 non-null  float64       
 10  Newspaper                    990262 non-null  object        
 11  RegSentsExpand            

In [19]:
print(df3[df3['TotalWordCount']==0][['ID','RegSentsExpand','NounChunkMatchFiltered']])

                ID RegSentsExpand  NounChunkMatchFiltered
381      294314085                                      0
559      294345870                                      0
608      294249140                                      0
718      294290863                                      0
792      294247001                                      0
...            ...            ...                     ...
989977  2610985065                                      0
989998  2611050480                                      0
990062  2612064676                                      0
990079  2612463089                                      0
990248  2615508225                                      0

[46496 rows x 3 columns]


In [14]:
# Export all data
df3.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Jan1985-Dec2021/RegRelevant_ArticleSentimentWordsScores.pkl')

In [15]:
# Export sentiment scores only
sentimentScores=df3[['ID','StartDate','Newspaper','PubTitle','UncertaintyScore','GIscore','LMscore','LSDscore']]
print(sentimentScores.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990262 entries, 0 to 990261
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                990262 non-null  object        
 1   StartDate         990262 non-null  datetime64[ns]
 2   Newspaper         990262 non-null  object        
 3   PubTitle          990262 non-null  object        
 4   UncertaintyScore  943766 non-null  float64       
 5   GIscore           943766 non-null  float64       
 6   LMscore           943766 non-null  float64       
 7   LSDscore          943766 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 68.0+ MB
None


In [16]:
sentimentScores.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Jan1985-Dec2021/RegRelevant_ArticleSentimentScores.csv',index=False)